In [1]:
import numpy as np

def lu(A):
    
    #Get the number of rows
    n = A.shape[0]
    # inititalize LU factors
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    
    #Loop over rows
    for i in range(n):
        print("i =", i)
        print("L: \n", L)
        print("U: \n", U)
        #Eliminate entries below i with row operations 
        #on U and reverse the row operations to 
        #manipulate L
        factor = U[i+1:, i] / U[i, i]
        print("column factor", factor)
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]
    
    return L, U

In [44]:
A = np.array([[5, 2, 1], [5, -6, 2], [-4, 2, 1]]).astype(np.float64)
print(A)
L, U = lu(A)

[[ 5.  2.  1.]
 [ 5. -6.  2.]
 [-4.  2.  1.]]
i = 0
L: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
U: 
 [[ 5.  2.  1.]
 [ 5. -6.  2.]
 [-4.  2.  1.]]
column factor [ 1.  -0.8]
i = 1
L: 
 [[ 1.   0.   0. ]
 [ 1.   1.   0. ]
 [-0.8  0.   1. ]]
U: 
 [[ 5.   2.   1. ]
 [ 0.  -8.   1. ]
 [ 0.   3.6  1.8]]
column factor [-0.45]
i = 2
L: 
 [[ 1.    0.    0.  ]
 [ 1.    1.    0.  ]
 [-0.8  -0.45  1.  ]]
U: 
 [[ 5.    2.    1.  ]
 [ 0.   -8.    1.  ]
 [ 0.    0.    2.25]]
column factor []


In [45]:
A = np.array([[1, 1, 0], [2, 1, -1], [3, -1, -1]]).astype(np.float64)
print(A)
L, U = lu(A)
#
#print(U)
#print(np.dot(L,U))

[[ 1.  1.  0.]
 [ 2.  1. -1.]
 [ 3. -1. -1.]]
i = 0
L: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
U: 
 [[ 1.  1.  0.]
 [ 2.  1. -1.]
 [ 3. -1. -1.]]
column factor [2. 3.]
i = 1
L: 
 [[1. 0. 0.]
 [2. 1. 0.]
 [3. 0. 1.]]
U: 
 [[ 1.  1.  0.]
 [ 0. -1. -1.]
 [ 0. -4. -1.]]
column factor [4.]
i = 2
L: 
 [[1. 0. 0.]
 [2. 1. 0.]
 [3. 4. 1.]]
U: 
 [[ 1.  1.  0.]
 [ 0. -1. -1.]
 [ 0.  0.  3.]]
column factor []


In [46]:
def forward_substitution(L, b):
    #Get number of rows
    n = L.shape[0]
    #Allocating space for the solution vector
    y = np.zeros_like(b)
    #Here we perform the forward-substitution.  
    #Initializing  with the first row.
    y[0] = b[0] / L[0, 0]
    #Looping over rows in reverse (from the bottom  up),
    #starting with the second to last row, because  the 
    #last row solve was completed in the last step.
    for i in range(1, n):
        print(L[i,:i]) # all the line i up to the column i, that is the subdiagonal line
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
    return y

print(L)
print("forward substitution")
b = np.ones(L.shape[0])
y = forward_substitution(L, b)
print("result")
print(y)

[[1. 0. 0.]
 [2. 1. 0.]
 [3. 4. 1.]]
forward substitution
[2.]
[3. 4.]
result
[ 1. -1.  2.]


In [47]:
def plu(A):
    """
    Implementation of a pivoted LU decomposition.
    
    Returns matrices P, L, U, such that
    PA = LU.
    """
    
    import numpy as np
    
    # Return an error if matrix is not square
    if not A.shape[0]==A.shape[1]:
        raise ValueError("Input matrix must be square")
        
    n = A.shape[0] # The number of rows/columns of A
    p = np.arange(n) # The permutation vector
    
    L = np.zeros((n,n),dtype='float64') # Reserve space for L
    U = np.zeros((n,n),dtype='float64') # Reserve space for U
    U[:] = A # Copy A into U as we do not want to modify A
    np.fill_diagonal(L,1) # fill the diagonal of L with 1
    
    for i in range(n-1):
        # The outer iteration
        # Permute the rows to ensure the element with largest magnitude is on top
        max_index = i+np.argmax(np.abs(U[i:,i]))
        U[[i,max_index],:] = U[[max_index,i],:]
        L[[i,max_index],:i] = L[[max_index,i],:i]
        p[[i,max_index]] = p[[max_index,i]]
        for j in range(i+1,n):
            L[j,i] = U[j,i]/U[i,i]
            U[j,i:] = U[j,i:]-L[j,i]*U[i,i:]
            U[j,i] = 0
    P = np.eye(n)[p,:] 
    return (P,L,U)

In [48]:
print(A)
P, L, U = plu(A)
print("permutation matrix")
print(P)
print("lower")
print(L)
print("upper")
print(U)

[[ 1.  1.  0.]
 [ 2.  1. -1.]
 [ 3. -1. -1.]]
permutation matrix
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
lower
[[1.         0.         0.        ]
 [0.66666667 1.         0.        ]
 [0.33333333 0.8        1.        ]]
upper
[[ 3.         -1.         -1.        ]
 [ 0.          1.66666667 -0.33333333]
 [ 0.          0.          0.6       ]]
